# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data/weather_data_refined.csv')

weather_df

,City,Country,Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,rikitea,PF,78.98,70,99,7.45,-23.1203,-134.9692
1,punta arenas,CL,53.29,54,75,19.57,-53.1500,-70.9167
2,tiksi,RU,-27.76,84,74,7.20,71.6872,128.8694
3,kuryk,KZ,17.08,87,88,22.35,43.2000,51.6500
4,kapaa,US,71.47,78,75,17.27,22.0752,-159.3190
...,...,...,...,...,...,...,...,...
551,pokrovsk,UA,11.57,89,0,12.41,48.2820,37.1758
552,akureyri,IS,30.20,93,75,9.22,65.6835,-18.0878
553,leshukonskoye,RU,-8.70,90,28,3.13,64.8989,45.7622
554,rio gallegos,AR,55.40,41,0,14.97,-51.6226,-69.2181


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_df[['Latitude','Longitude']]
humidity = weather_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, point_radius=3, dissipating=False, max_intensity=100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:

weather_df = weather_df.rename(columns={'Temperature': 'Temperature (F)','Humidity': "Humidity (%)",'Cloudiness': 'Cloudiness (%)','Wind Speed': 'Wind Speed (mph)'})
weather_df.head()
weather_df.dtypes

City                 object
Country              object
Temperature (F)     float64
Humidity (%)          int64
Cloudiness (%)        int64
Wind Speed (mph)    float64
Latitude            float64
Longitude           float64
dtype: object

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_df = weather_df[(weather_df['Temperature (F)'] > 70) & (weather_df['Temperature (F)'] <80)]
weather_df = weather_df[weather_df['Wind Speed (mph)'] < 10]
weather_df = weather_df[weather_df['Cloudiness (%)'] == 0]

In [6]:
weather_df['Hotel Name'] = ""
weather_df

,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Latitude,Longitude,Hotel Name
19,lima,PE,74.37,69,0,6.91,-12.0432,-77.0282,
26,mar del plata,AR,73.65,40,0,8.05,-38.0023,-57.5575,
52,hermanus,ZA,73.99,67,0,4.00,-34.4187,19.2345,
119,ambagarh chauki,IN,72.46,17,0,2.13,20.7833,80.7167,
120,nueva helvecia,UY,79.00,46,0,8.05,-34.3000,-57.2333,
181,salalah,OM,77.00,69,0,5.75,17.0151,54.0924,
258,pisco,PE,73.65,68,0,6.91,-13.7000,-76.2167,
317,parvatsar,IN,72.66,20,0,6.67,26.8833,74.7667,
330,basavakalyan,IN,76.42,19,0,9.73,17.8667,76.9500,
369,basmat,IN,78.67,18,0,4.61,19.3167,77.1667,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in weather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params).json()
    
    try:
        weather_df.loc[index, "Hotel Name"] = name_address['results'][0]["name"]
    except:
        print("error")

In [8]:
hotel_df = weather_df
hotel_df

,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Latitude,Longitude,Hotel Name
19,lima,PE,74.37,69,0,6.91,-12.0432,-77.0282,Gran Hotel Bolivar Lima
26,mar del plata,AR,73.65,40,0,8.05,-38.0023,-57.5575,Gran Hotel Mar del Plata
52,hermanus,ZA,73.99,67,0,4.00,-34.4187,19.2345,Misty Waves Boutique Hotel
119,ambagarh chauki,IN,72.46,17,0,2.13,20.7833,80.7167,गाँधी चौक शांति भवन सोनसाय टोला
120,nueva helvecia,UY,79.00,46,0,8.05,-34.3000,-57.2333,La Posada de Ofelia
181,salalah,OM,77.00,69,0,5.75,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
258,pisco,PE,73.65,68,0,6.91,-13.7000,-76.2167,Hotel San Isidro Oficial
317,parvatsar,IN,72.66,20,0,6.67,26.8833,74.7667,Shree Hotel
330,basavakalyan,IN,76.42,19,0,9.73,17.8667,76.9500,Tourist Lodge
369,basmat,IN,78.67,18,0,4.61,19.3167,77.1667,Sai Hotal


In [36]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [58]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(locations, scale=2, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', info_box_content=hotel_name)


# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))